### Simulator

In [1]:
import numpy as np
import pandas as pd
import random
import math
from matplotlib import pyplot as plt

In [2]:
# warmup period approach 1 and 2
# all_num_busy_channels = [] # stores sum of busy channels for 100 replications
# all_count = [] # stores number of events handled in 1 hour for 100 replications

# data collection for paired-t confidence interval

# no channel reservation
nores_blocked_calls_percentages = [] 
nores_dropped_calls_percentages = []

# 1 channel reservation
res_blocked_calls_percentages = [] 
res_dropped_calls_percentages = []

# 2 channels reservation
res2_blocked_calls_percentages = [] 
res2_dropped_calls_percentages = []

In [10]:
def main():
    
    #######################################

    # declaring global variables
    global clock
    global busy_channels
    global fel
    
    global initiated_calls
    global blocked_calls
    global dropped_calls
    
    # initialising global variables
    clock = 0
    busy_channels = np.zeros(20).astype(int)
    fel = []
    
    initiated_calls = 0
    blocked_calls = 0
    dropped_calls = 0
    
    #######################################
    
    # generating variables of the first call
    initiation_time, duration, station, position, direction, speed = generate_call_variables()
    
    # create initiation event for first call
    event = {"type" : "initiation",
             "time" : initiation_time,
             "duration" : duration,
             "station" : station,
             "position" : position,
             "direction" : direction,
             "speed" : speed}
    
    # add event to FEL and sort FEL
    fel.append(event)
    fel = sorted(fel, key = lambda d: d["time"])
    
    #######################################
    
    '''
    # metrics to determine warmup period
    global num_busy_channels # stores sum of busy channels for 1 replication
    global all_num_busy_channels # stores sum of busy channels for 100 replications
    
    num_busy_channels = []
    '''

    #######################################
    
    '''
    # warmup period approach 1
    count = 0 # number of events handled
    
    while clock < 3600: # set clock to run for 1 hour
        event = fel.pop(0)
        clock = event["time"]
        run_event(event)
        num_busy_channels.append(sum(busy_channels))
        count += 1
    
    all_num_busy_channels.append(num_busy_channels)
    all_count.append(count)
    '''
    
    #######################################
    
    '''
    # warmup period approach 2
    count = 0 # number of events handled
    
    while count < 9259: # set number of events handled to 9268 (no channel reservation) / 9259 (channel reservation)
        event = fel.pop(0)
        clock = event["time"]
        run_event(event)
        num_busy_channels.append(sum(busy_channels))
        count += 1
    
    all_num_busy_channels.append(num_busy_channels)
    '''
    
    #######################################
    
    # main body
    count = 0
    
    while initiated_calls < 20000: # terminates after handling 20,000 intiated calls
        
        if count == 800: # drop data after 1000 events (no channel reservation) / 800 events (channel reservation)
            initiated_calls = 0
            blocked_calls = 0
            dropped_calls = 0
            
        event = fel.pop(0)
        clock = event["time"]
        run_event(event)
        count += 1
        
    #######################################
    
    # performance metrics
    blocked_calls_percentage = round((blocked_calls / initiated_calls) * 100, 3)
    dropped_calls_percentage = round((dropped_calls / initiated_calls) * 100, 3)
    
    # adding percentages to master lists (COMMENT OUT THE OTHER FCA SCHEME WHEN NOT IN USE)
    
    '''
    # no channel reservation
    nores_blocked_calls_percentages.append(blocked_calls_percentage)
    nores_dropped_calls_percentages.append(dropped_calls_percentage)
    '''
    
    # channel reservation
    res_blocked_calls_percentages.append(blocked_calls_percentage)
    res_dropped_calls_percentages.append(dropped_calls_percentage)
    # '''
    
    # 2 channel reservation
    res2_blocked_calls_percentages.append(blocked_calls_percentage)
    res2_dropped_calls_percentages.append(dropped_calls_percentage)
    # '''
    
    #######################################
    
    # statistical report
    hours = int(clock / 3600)
    minutes = int((clock % 3600) / 60)
    seconds = int((clock % 3600) % 60)
    
    print()
    print("************************************************************")
    print("******************** Statistical Report ********************")
    print()
    print("Physical Time Elapsed = " + str(hours) + " hours " + str(minutes) + " minutes " + str(seconds) + " seconds")
    print()
    print("Number of Initiated Calls = " + str(initiated_calls))
    print("Number of Blocked Calls = " + str(blocked_calls))
    print("Number of Dropped Calls = " + str(dropped_calls))
    print()
    print("Percentage of Blocked Calls = " + str(blocked_calls_percentage))
    print("Percentage of Dropped Calls = " + str(dropped_calls_percentage))
    print()
    print("************************************************************")
    print("************************************************************")
    
    return None

In [11]:
def generate_call_variables():
    
    # referencing global variables
    global clock

    #######################################

    # exponentially distributed parameters

    # inter-arrival time (seconds)
    INTERARRIVAL_BETA = 1.370

    # call duration (seconds)
    DURATION_BETA = 99.832

    #######################################

    # uniformly distributed parameters

    # base station
    BASE_STN_A = 0
    BASE_STN_B = 20

    # car position (m)
    POSITION_A = 0
    POSITION_B = 2000 # 2 km

    # car direction
    LEFT = 0
    RIGHT = 1

    #######################################

    # normally distributed parameters

    # car speed (m/s)
    SPEED_MEAN = 33.353 # 120.072 km/h
    SPEED_STD = 2.505 # 9.019 km/h

    #######################################
    
    interarrival_time = np.random.exponential(INTERARRIVAL_BETA)
    initiation_time = clock + interarrival_time
    duration = np.random.exponential(DURATION_BETA) + 10.004
    
    station = np.random.randint(BASE_STN_A, BASE_STN_B)
    position = np.random.uniform(POSITION_A, POSITION_B)
    direction = np.random.randint(LEFT, RIGHT + 1)
    
    speed = np.random.normal(SPEED_MEAN, SPEED_STD)
    
    return initiation_time, duration, station, position, direction, speed

In [12]:
def run_event(event):
    
    if event["type"] == "initiation":
        initiation_event_handler(event["time"], event["duration"], event["station"],
                               event["position"], event["direction"], event["speed"])
        
    elif event["type"] == "handover":
        handover_event_handler(event["time"], event["duration"], event["station"],
                             event["direction"], event["speed"])
    
    elif event["type"] == "termination":
         termination_event_handler(event["time"], event["station"])
            
    else:
        print("Wrong event type detected: " + event["type"])
        
    return None

In [13]:
def initiation_event_handler(time, duration, station, position, direction, speed):
    
    # referencing global variables
    global clock
    global busy_channels
    global fel
    
    global initiated_calls
    global blocked_calls
    global dropped_calls
    
    initiated_calls += 1
    
    # generating variables of the next call
    next_initiation_time, next_duration, next_station, next_position, next_direction, next_speed = generate_call_variables()
    
    # create initiation event for next call
    event = {"type" : "initiation",
             "time" : next_initiation_time,
             "duration" : next_duration,
             "station" : next_station,
             "position" : next_position,
             "direction" : next_direction,
             "speed" : next_speed}
    
    # add event to FEL and sort FEL
    fel.append(event)
    fel = sorted(fel, key = lambda d: d["time"])
    
    # if number of busy cell channels is 9 or 10, block the call
    if busy_channels[station] >= 8: # == 10 for no channel reservation, >= 9 for channel reservation, >= 8 for 2 channel reservation
        blocked_calls = blocked_calls + 1
        
    else:
        busy_channels[station] += 1
        
        # calculating distance left to end of cell
        if direction == 0:
            remaining_cell_distance = position
            
        else:
            remaining_cell_distance = 2000 - position
        
        # distance to be travelled by car
        total_distance = speed * duration
        
        # if call will not continue after current station, prepare for termination
        if total_distance <= remaining_cell_distance:
            
            # create termination event
            termination_time = clock + duration
            event = {"type" : "termination",
                     "time" : termination_time,
                     "station" : station}
    
            # add event to FEL and sort FEL
            fel.append(event)
            fel = sorted(fel, key = lambda d: d["time"])
        
        else:
            # if call is in last station, prepare for termination. Else, handover.
            if direction == 0:
                last_station = 0
                
            else:
                last_station = 19
                
            if station == last_station:
                
                # create termination event
                termination_time = clock + remaining_cell_distance / speed
                event = {"type" : "termination",
                         "time" : termination_time,
                         "station" : station}
    
                # add event to FEL and sort FEL
                fel.append(event)
                fel = sorted(fel, key = lambda d: d["time"])
                
            else:
                # create handover event
                handover_time = clock + remaining_cell_distance / speed
                duration = duration - remaining_cell_distance / speed
                event = {"type" : "handover",
                         "time" : handover_time,
                         "duration" : duration,
                         "station" : station,
                         "direction" : direction,
                         "speed" : speed}
    
                # add event to FEL and sort FEL
                fel.append(event)
                fel = sorted(fel, key = lambda d: d["time"])
                
    return None

In [14]:
def handover_event_handler(time, duration, station, direction, speed):
    
    # referencing global variables
    global clock
    global busy_channels
    global fel
    
    global initiated_calls
    global blocked_calls
    global dropped_calls
    
    busy_channels[station] -= 1;
    
    # update station
    if direction == 0:
        station -= 1
    
    else:
        station += 1
        
    # if number of busy cell channels in newStation is 10, drop the call
    if busy_channels[station] == 10:
        dropped_calls = dropped_calls + 1
    
    else:
        busy_channels[station] += 1
        
        # remaining distance to be travelled by car
        total_distance = speed * duration
        
        # if call will not continue after current station, prepare for termination
        if total_distance <= 2000:
            
            # create termination event
            termination_time = clock + duration
            event = {"type" : "termination",
                     "time" : termination_time,
                     "station" : station}
    
            # add event to FEL and sort FEL
            fel.append(event)
            fel = sorted(fel, key = lambda d: d["time"])
            
        else:
            # if call is in last station, prepare for termination. Else, handover.
            if direction == 0:
                last_station = 0
                
            else:
                last_station = 19
                
            if station == last_station:
                
                # create termination event
                termination_time = clock + 2000 / speed
                event = {"type" : "termination",
                         "time" : termination_time,
                         "station" : station}
    
                # add event to FEL and sort FEL
                fel.append(event)
                fel = sorted(fel, key = lambda d: d["time"])
                
            else:
                # create handover event
                handover_time = clock + 2000 / speed
                duration = duration - 2000 / speed
                event = {"type" : "handover",
                         "time" : handover_time,
                         "duration" : duration,
                         "station" : station,
                         "direction" : direction,
                         "speed" : speed}
    
                # add event to FEL and sort FEL
                fel.append(event)
                fel = sorted(fel, key = lambda d: d["time"])
    
    return None

In [15]:
def termination_event_handler(time, station):
    
    # referencing global variables
    global busy_channels
    
    busy_channels[station] -= 1
    
    return None

In [16]:
for i in range(100):
    main()


************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 45 minutes 35 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 227
Number of Dropped Calls = 58

Percentage of Blocked Calls = 1.135
Percentage of Dropped Calls = 0.29

************************************************************
************************************************************

************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 38 minutes 26 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 240
Number of Dropped Calls = 65

Percentage of Blocked Calls = 1.2
Percentage of Dropped Calls = 0.325

************************************************************
************************************************************

************************************************************
*****


************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 46 minutes 28 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 204
Number of Dropped Calls = 85

Percentage of Blocked Calls = 1.02
Percentage of Dropped Calls = 0.425

************************************************************
************************************************************

************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 36 minutes 49 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 237
Number of Dropped Calls = 70

Percentage of Blocked Calls = 1.185
Percentage of Dropped Calls = 0.35

************************************************************
************************************************************

************************************************************
****


************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 42 minutes 16 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 234
Number of Dropped Calls = 84

Percentage of Blocked Calls = 1.17
Percentage of Dropped Calls = 0.42

************************************************************
************************************************************

************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 43 minutes 57 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 223
Number of Dropped Calls = 66

Percentage of Blocked Calls = 1.115
Percentage of Dropped Calls = 0.33

************************************************************
************************************************************

************************************************************
*****


************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 42 minutes 32 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 219
Number of Dropped Calls = 66

Percentage of Blocked Calls = 1.095
Percentage of Dropped Calls = 0.33

************************************************************
************************************************************

************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 42 minutes 16 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 223
Number of Dropped Calls = 69

Percentage of Blocked Calls = 1.115
Percentage of Dropped Calls = 0.345

************************************************************
************************************************************

************************************************************
***


************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 41 minutes 22 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 225
Number of Dropped Calls = 68

Percentage of Blocked Calls = 1.125
Percentage of Dropped Calls = 0.34

************************************************************
************************************************************

************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 41 minutes 53 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 234
Number of Dropped Calls = 81

Percentage of Blocked Calls = 1.17
Percentage of Dropped Calls = 0.405

************************************************************
************************************************************

************************************************************
****


************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 43 minutes 28 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 241
Number of Dropped Calls = 53

Percentage of Blocked Calls = 1.205
Percentage of Dropped Calls = 0.265

************************************************************
************************************************************

************************************************************
******************** Statistical Report ********************

Physical Time Elapsed = 7 hours 45 minutes 33 seconds

Number of Initiated Calls = 20000
Number of Blocked Calls = 210
Number of Dropped Calls = 67

Percentage of Blocked Calls = 1.05
Percentage of Dropped Calls = 0.335

************************************************************
************************************************************

************************************************************
***

In [ ]:
# warmup period approach 1
# avg_count = sum(all_count) / len(all_count)
# print("Average Number of Events Handled in 1 Hour = " + str(avg_count))

In [ ]:
# warmup period approach 2
# df = pd.DataFrame.from_records(all_num_busy_channels).T
# df

In [ ]:
# warmup period approach 2
# df["Average Sum of Busy Channels over 100 runs"] = df.mean(axis = 1)
# df

In [ ]:
# warmup period approach 2
# plt.plot(df["Average Sum of Busy Channels over 100 runs"])
#plt.axvline(x = 800, ymin = 0, ymax = 80, color = 'r')

# plt.xlabel("Number of Events Handled")
# plt.ylabel("Sum of Busy Channels (average over 100 runs)")

# plt.show()

In [17]:
# RUN ONLY AFTER BOTH FCA SCHEMES ARE USED

#######################################

# paired-t confidence interval
t_90 = 1.660
t_95 = 1.984
t_99 = 2.626

#######################################

# blocked calls percentage

# calculating z = theta 1 - theta 2
array1 = np.array(nores_blocked_calls_percentages)
array2 = np.array(res_blocked_calls_percentages)

subtracted_array = np.subtract(array1, array2)
z = list(subtracted_array)

# calculating mean and variance
blocked_mean = sum(z) / len(z)
blocked_var = np.var(z)

# calculating confidence interval

blocked_lower_90 = round(blocked_mean - t_90 * math.sqrt(blocked_var / len(z)), 3)
blocked_upper_90 = round(blocked_mean + t_90 * math.sqrt(blocked_var / len(z)), 3)

blocked_lower_95 = round(blocked_mean - t_95 * math.sqrt(blocked_var / len(z)), 3)
blocked_upper_95 = round(blocked_mean + t_95 * math.sqrt(blocked_var / len(z)), 3)

blocked_lower_99 = round(blocked_mean - t_99 * math.sqrt(blocked_var / len(z)), 3)
blocked_upper_99 = round(blocked_mean + t_99 * math.sqrt(blocked_var / len(z)), 3)

#######################################

# dropped calls percentage

# calculating z = theta 1 - theta 2
array1 = np.array(nores_dropped_calls_percentages)
array2 = np.array(res_dropped_calls_percentages)

subtracted_array = np.subtract(array1, array2)
z = list(subtracted_array)

# calculating mean and variance
dropped_mean = sum(z) / len(z)
dropped_var = np.var(z)

# calculating confidence interval

dropped_lower_90 = round(dropped_mean - t_90 * math.sqrt(dropped_var / len(z)), 3)
dropped_upper_90 = round(dropped_mean + t_90 * math.sqrt(dropped_var / len(z)), 3)

dropped_lower_95 = round(dropped_mean - t_95 * math.sqrt(dropped_var / len(z)), 3)
dropped_upper_95 = round(dropped_mean + t_95 * math.sqrt(dropped_var / len(z)), 3)

dropped_lower_99 = round(dropped_mean - t_99 * math.sqrt(dropped_var / len(z)), 3)
dropped_upper_99 = round(dropped_mean + t_99 * math.sqrt(dropped_var / len(z)), 3)

#######################################

print("**********************************************************************")
print("**************** FCA Scheme 1: No Channel Reservation ****************")
print("**********************************************************************")
print()
print("Mean Percentage of Blocked Calls = " + str(round(sum(nores_blocked_calls_percentages) / len(nores_blocked_calls_percentages), 3)))
print("Mean Percentage of Dropped Calls = " + str(round(sum(nores_dropped_calls_percentages) / len(nores_dropped_calls_percentages), 3)))
print()
print("**********************************************************************")
print("*************** FCA Scheme 2: With Channel Reservation ***************")
print("**********************************************************************")
print()
print("Mean Percentage of Blocked Calls = " + str(round(sum(res_blocked_calls_percentages) / len(res_blocked_calls_percentages), 3)))
print("Mean Percentage of Dropped Calls = " + str(round(sum(res_dropped_calls_percentages) / len(res_dropped_calls_percentages), 3)))
print()
print("**********************************************************************")
print("******************** t-paired Confidence Interval ********************")
print("**********************************************************************")
print()
print("********************** Blocked Calls Percentage **********************")
print()
print("Mean = " + str(blocked_mean))
print("Variance = " + str(blocked_var))
print()
print("90% Confidence Interval = (" + str(blocked_lower_90) + ", " + str(blocked_upper_90) + ")")
print("95% Confidence Interval = (" + str(blocked_lower_95) + ", " + str(blocked_upper_95) + ")")
print("99% Confidence Interval = (" + str(blocked_lower_99) + ", " + str(blocked_upper_99) + ")")
print()
print("********************** Dropped Calls Percentage **********************")
print()
print("Mean = " + str(dropped_mean))
print("Variance = " + str(dropped_var))
print()
print("90% Confidence Interval = (" + str(dropped_lower_90) + ", " + str(dropped_upper_90) + ")")
print("95% Confidence Interval = (" + str(dropped_lower_95) + ", " + str(dropped_upper_95) + ")")
print("99% Confidence Interval = (" + str(dropped_lower_99) + ", " + str(dropped_upper_99) + ")")
print()
print("**********************************************************************")
print("**********************************************************************")
print("**********************************************************************")

**********************************************************************
**************** FCA Scheme 1: No Channel Reservation ****************
**********************************************************************

Mean Percentage of Blocked Calls = 0.345
Mean Percentage of Dropped Calls = 0.582

**********************************************************************
*************** FCA Scheme 2: With Channel Reservation ***************
**********************************************************************

Mean Percentage of Blocked Calls = 1.105
Mean Percentage of Dropped Calls = 0.346

**********************************************************************
******************** t-paired Confidence Interval ********************
**********************************************************************

********************** Blocked Calls Percentage **********************

Mean = -0.7598499999999996
Variance = 0.014559727500000001

90% Confidence Interval = (-0.78, -0.74)
95% Confidence Inte